# Notebook 5

Monte Carlo simulation is a flexible tool for pricing options with exotic payoffs including path-dependent options. It however tends to be rather inaccurate for a smaller number of simulations. Variance reduction techniques can help improve the accuracy of the method. In this tutorial we will use variance reduction techniques like the antithetic variables method, the control variates technique or importance sampling both in the Black-Scholes and in the Heston model. In order to price path-dependent options, we will employ both the Euler and Milshtein simulation scheme but also bias-free simulation methods.

In the code snippet below, we define the variables $r$, $\sigma$, $T$, $K$ and $S(0)$ for the Black-Scholes model and the variables $\gamma_0$, $\kappa$, $\lambda$ and $\widetilde \sigma$ for the Heston model.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, norminvgauss
from scipy.optimize import minimize

r = 0.02
sigma = 0.3
gam0 = 0.3**2
kappa = 0.3**2
lamb = 2.5
sig_tilde = 0.3
T = 1
K = 100
S0 = 120

<br><br>

## Step 1: Pricing European options via Monte Carlo simulation with antithetic variables

The antithetic variables method is a simple approach for reducing the variance of a Monte Carlo estimator by doubling the number of considered paths in a special way. We will first start by computing a regular Monte Carlo estimator for the price of a European call option in the Black-Scholes model using $N = 10\,000$ simulations.

**Task 1.1**: Define $N = 10\,000$. Then, draw $N$ samples of the standard normal distribution and store them in an array `X`. Afterwards, compute the $N = 10\,000$ discounted payoffs \begin{equation} e^{-rT} \big( S(0) e^{(r - \sigma^2 / 2) T + \sigma \sqrt{T} X_n} - K\big)^+\end{equation}
and store them in an array `disc_payoffs`. Use the numpy random seed 20 for reproducibility.

*Useful Python commands*: `np.random.normal`, `np.exp`, `np.maximum`, `np.sqrt`

In [ ]:
N = 10000

...
X = ...
disc_payoffs = ...

**Task 1.2**: Use the array `disc_payoffs` to compute the Monte Carlo estimate $\widehat V_N$ for the European call option price $V(0)$ in the Black-Scholes model model and store it in a variable `call_BS_MC`. Then, define a variable `epsilon_95` that contains the width $\widehat \varepsilon_N := \smash{1.96 \sqrt{\frac{\widehat \sigma^2_N(f(X))}{N}}}$ of an approximate 95\% confidence interval for $V(0)$, where $\widehat \sigma^2_N(f(X))$ denotes the sample variance of the discounted payoffs from the array `disc_payoffs`. Then, print the variables `call_BS_MC` and `epsilon_95` as well as the relative percentage error $100\frac{\widehat \varepsilon_N}{\widehat V_N}$.

*Useful Python commands*: `np.mean`, `np.var` `np.sqrt`

In [ ]:
call_BS_MC = ...
epsilon_95 = ...

print('MC option price: {:.5f}, 95% error: {:.5f} ({:.5f}%)'.format(call_BS_MC, epsilon_95, 100 * epsilon_95 / call_BS_MC))

We will now perform the antithetic variables method and will examine by how much the variance of the Monte Carlo estimator can be reduced by it.

**Task 1.3**: Define an array `disc_payoffs_anti` that contains the $N = 10\,000$ discounted payoffs $e^{-rT} (S(T) - K)^+$. Use the same formula as in Task 1.1 to compute the payoffs but instead of using the Gaussian simulations $X_n$, use the simulations $-X_n$. Then define an array `disc_payoffs_average` that contains the elementwise arithmetic average of the payoffs from `disc_payoffs` and `disc_payoffs_anti`.

*Useful Python commands*: `np.maximum`

In [ ]:
disc_payoffs_anti = ...
disc_payoffs_average = ...

**Task 1.4**: Repeat Task 1.2 but this time, use the array `disc_payoffs_average` instead of `disc_payoffs`. Store the resulting antithetic variables estimator and its 95% confidence interval width in variables `call_BS_MC_anti` and `epsilon_95_anti`. Again print them as well as the corresponding relative percentage error and interpret your result in comparison to the result from Task 1.2.

In [ ]:
call_BS_MC_anti = ...
epsilon_95_anti = ...

print('Antithetic MC option price: {:.4f}, 95% error: {:.4f} ({:.4f}%)'.format(call_BS_MC_anti, epsilon_95_anti, 100 * epsilon_95_anti / call_BS_MC_anti))

We now want to visualize how much faster the antithetic variables Monte Carlo estimators converge to the true price of the European call option in the Black-Scholes model as the number $N$ of simulations grows. We first compute the true Black-Scholes price using the Black-Scholes formula.

**Task 1.5**: Copy your function `european_BS(t, St, K, T, r, sigma, call)` that computes European option prices in the Black-Scholes model from Task 1.5 of the Week 03 notebook, or write it from scratch if you did not complete that task from week 03. Then, use this function to define a variable `call_BS` that contains the price $V(0)$ of the European call option with strike $K = 100$ and all other variables as set in the beginning of the notebook.

In [ ]:
...

call_BS = ...

**Task 1.6**: Define an array `call_BS_MC_path` of length $10\,000$. The $n$-th entry of this array should contain the Monte Carlo estimator for the European call option price using only the first $n$ simulations from the array `disc_payoffs`. Then, define an array `call_BS_MC_path_anti` of length $10\,000$. The $n$-th entry of this array should contain the antithetic variables Monte Carlo estimator for the European call option price using only the first $n$ simulations from the array `disc_payoffs_average`. 

*Useful Python commands*: `np.cumsum`

In [ ]:
...

**Task 1.7**: Plot the Monte Carlo estimators from the array `call_BS_MC_path` against the number $n$ of simulations, starting from $n = 100$. In the same plot window, plot the antithetic variables Monte Carlo estimators from the array `call_BS_MC_path_anti` against the number $n$ of simulations, starting from $n = 100$. Draw a horizontal line at the height of the true option price stored in the variable `call_BS`. Interpret your result using a comment.

*Useful Python commands*: `plt.axhline`

In [ ]:
...
plt.show()

<br>

## Step 2: Simulation of a geometric Brownian motion via the Euler and Milshtein schemes

For the valuation of path-dependent options it is necessary to simulate not only the terminal value $S(T)$ of the stock price path (as it was done in Step 1) but to simulate the whole path of the stock price $S(t)$, $t \in [0, T]$. Classical simulation schemes for stochastic integrals like the Euler scheme will sample $N$ discretized paths $S(t_0), S(t_1), \dots, S(t_m)$ with $0 = t_0 < t_1 < \dots < t_m = T$.

**Task 2.1**: Let $N = 10\,000$ and let $m = 1000$ be the number of steps per path, i.e. $\Delta T = T / m = T / 1000$. Initialize an array `dWt` of shape `(N, m)` for the increments of the Brownian motions $W$ containing independent normal random variables with mean 0 and standard deviation $\sqrt{\Delta t}$. Use the numpy random seed 20 to ensure reproducibility of your results.

*Useful Python commands*: `np.random.normal`

In [ ]:
N = 10000
m = 1000
dt = T / m

...
dWt = ...

**Task 2.2**: Initialize an array `S_Euler` of shape `(N, m + 1)` containing only zeros, which are later supposed to be filled with the $S$ paths. Set the elements of the first column of `S` equal to `S0`.

*Useful Python commands*: `np.zeros`

In [ ]:
S_Euler = ...
S_Euler[:, 0] = ...

**Task 2.3**: Use a for-loop over the columns of `S` to compute the paths of the stock $S$ using the Euler method.

*Useful Python commands*: `np.sqrt`

In [ ]:
for i in range(m):
    S_Euler[:, i + 1] = ...

After having simulated stock price paths using the Euler method, we want to use the more refined Milshtein method, which has a smaller discretization bias, to simulate another set of stock price paths. 

**Task 2.4**: Repeat tasks 2.2 and 2.3 but this time, use the Milshtein scheme to simulate the stock price paths. To make the simulated paths comparable to those from Task 2.3, use the same simulations for the Brownian motion increments (stored in the array `dWt`) as used for the Euler method. Store your paths in an array `S_Milshtein` of shape `(N, m + 1)`.

In [ ]:
S_Milshtein = ...
S_Milshtein[:, 0] = ...

for i in range(m):
    S_Milshtein[:, i + 1] = ...

**Task 2.5**: Plot the first simulated path from the array `S_Euler` and from the `S_Milshtein` in a common plot. In a separate plot window, plot the relative difference between those two paths as a fraction of the Milshtein path values and interpret your result using a comment.

In [ ]:
...
plt.show()

<br>

## Step 3: Valuation of Parisian options using Monte Carlo estimation with control variates

A *cumulative Parisian option* is a variant of a Barrier option in which the barrier condition (i.e. the knock-in or the knock-out) only triggers if the barrier is surpassed cumulatively over a so-called trigger period of length $\tau > 0$. More precisely, we would like to compute the price of a *cumulative Parisian down-and-out call option* with strike $K = 100$, barrier $B = 80$ and trigger period 1 month (i.e. $\tau = \frac{1}{12})$. This means that the option pays out a payoff of $(S(T) - K)^+$ at the terminal time $T$ as long as the total cumulative time the stock price $S(t)$ has stayed below the barrier $B$ is smaller than $\tau$. This means that the payoff of this Parisian option is given by 
\begin{equation} V(T) = \begin{cases} (S(T) - K)^+ & \text{ if } \;\int_0^T 1_{\{S(t) < B\}} dt < \tau, \\ 0 & \text{ else.}
\end{cases}
\end{equation}

As simulated stock price paths, we want to use the $N = 10\,000$ simulations from the Milshtein scheme simulated in Task 2.4.

**Task 3.1**: Define variables `B` and `tau` for $B = 80$ and $\tau = \frac{1}{12}$. Then, initialize an array `disc_payoffs` of length $10\,000$ that contains the discounted payoffs $e^{-rT} V(T)$ of the parisian option for all $N = 10\,000$ paths from the array `S_Milshtein`. The condition $\int_0^T 1_{\{S(t) < B\}} dt < \tau$ means that less than $\tau\cdot m$ discretized points $S(t_i)$ are less than $B$.

*Useful Python commands*: `np.maximum`, `np.sum`

In [ ]:
B = ...
tau = ...
disc_payoffs = ...

**Task 3.2**: Use the array `disc_payoffs` to compute the Monte Carlo estimate $\widehat V_N$ for the Parisian option price $V(0)$ in the Black-Scholes model model and store it in a variable `parisian_call_BS_MC`. Then, define a variable `var_MC` that contains the sample variance $\widehat \sigma^2_N(f(X))$ of the discounted payoffs from the array `disc_payoffs`. Finally, define a variable `epsilon_95` that contains the width $\widehat \varepsilon_N := \smash{1.96 \sqrt{\frac{\widehat \sigma^2_N(f(X))}{N}}}$ of an approximate 95\% confidence interval for $V(0)$. Print the variables `parisian_call_BS_MC` and `epsilon_95` as well as the relative percentage error $100\frac{\widehat \varepsilon_N}{\widehat V_N}$.

*Useful Python commands*: `np.mean`, `np.var` `np.sqrt`

In [ ]:
parisian_call_BS_MC = ...
var_MC = ...
epsilon_95 = ...

print('MC Parisian call option price: {:.4f}, 95% error: {:.4f} ({:.4f}%)'.format(parisian_call_BS_MC, epsilon_95, 100 * epsilon_95 / parisian_call_BS_MC))

We now want to reduce the 95% confidence error by using the control variates technique. A suitable candidate for a control variate for a Parisian down-and-out call is a regular down-and-out call because prices of regular down-and-out call options with barrier $B$ are known in closed form in the Black-Scholes model to be given by
\begin{equation} V_B(0) = S(0) \Big[\Phi(d_1) - \Big(\frac{B}{S(0)}\Big)^{\frac{2r}{\sigma^2} + 1} \Phi(m_1) \Big] - Ke^{-rT}  \Big[\Phi(d_2) - \Big(\frac{B}{S(0)}\Big)^{\frac{2r}{\sigma^2} - 1} \Phi(m_2) \Big]\end{equation}
with $d_1$ and $d_2$ as in (3.24) of the lecture notes and $m_1 = \frac{\log\big(\frac{B^2}{KS(0)}\big) + rT + \frac{\sigma^2}{2}T}{\sigma \sqrt{T}}$ and $m_2 = m_1 - \sigma \sqrt{T}$.

The code snippet below computes the time 0 price of a down-and-out call option with all parameters equal to those of the Parisian call option above.

In [ ]:
d1 = (np.log(S0 / K) + r * T + sigma**2 / 2 * T) / (sigma * np.sqrt(T))
d2 = (np.log(S0 / K) + r * T - sigma**2 / 2 * T) / (sigma * np.sqrt(T))
m1 = (np.log(B**2 / (K * S0)) + r * T + sigma**2 / 2 * T) / (sigma * np.sqrt(T))
m2 = (np.log(B**2 / (K * S0)) + r * T - sigma**2 / 2 * T) / (sigma * np.sqrt(T))
barrier_call_BS = S0 * (norm.cdf(d1) - (B / S0)**(1 + 2 * r / sigma**2) * norm.cdf(m1)) - K * np.exp(-r * T) * (norm.cdf(d2) - (B / S0)**(2 * r / sigma**2 - 1) * norm.cdf(m2))
print('Down-and-out call option price: {:.4f}'.format(barrier_call_BS))

To compute the control variates estimate for the Parisian option, we first have to find the Monte Carlo estimate for the regular barrier option.

**Task 3.3**: Define an array `disc_barrier_payoffs` that contains the $N = 10\,000$ discounted payoffs $e^{-rT}\big(S(T) - K\big)^+ 1_{\{\min_{i} S(t_i) \geq B\}}$ for $K = 100$ for all the simulated Milshtein paths from Step 1. Use the array `disc_barrier_payoffs` to compute the Monte Carlo estimate $\widehat V_N$ for the regular down-and-out call option price $V(0)$ in the Black-Scholes model and store it in a variable `barrier_call_BS_MC`.

*Useful Python commands*: `np.maximum`, `np.mean`, `np.exp`, `np.min`

In [ ]:
disc_barrier_payoffs = ...
barrier_call_BS_MC = ...

**Task 3.4**: Compute the optimal value $\beta$ from equation (5.8) in the lecture notes. In our case, $f(X)$ stands for the discounted Parisian payoffs and $Y$ stands for the discounted payoffs of the regular down-and-out call in (5.8). Then, compute the control variates estimate $V_{CV}$ for the Parisian option and store it in a variable `parisian_call_BS_MC_CV`.

*Useful Python commands*: `np.var`. The covariance between to arrays `arr1` and `arr2` can be computed as `np.cov(arr1, arr2)[0, 1]`.

In [ ]:
beta = ...
parisian_call_BS_MC_CV = ...

As detailed on page 56 of the lecture notes, the variance `var_MC` from Task 2.2 is reduced by $\mathrm{Cov}(f(X), Y)^2 / \mathrm{Var}(Y)$ by using the control variates estimate.

**Task 3.5**: Define a variable `epsilon_95_CV` that contains the width $\varepsilon_{CV}$ of an approximate 95\% confidence interval for $V_{CV}$. Print the variables `parisian_call_BS_MC_CV` and `epsilon_95_CV` as well as the relative percentage error $100\frac{\varepsilon_{CV}}{V_{CV}}$ and interpret your result in comparison to the result from Task 3.2.

*Useful Python commands*: `np.cov`, `np.var`, `np.sqrt`

In [ ]:
var_MC_CV = ...
epsilon_95_CV = ...

print('MC Parisian call option price with CV: {:.4f}, 95% error: {:.4f} ({:.4f}%)'.format(parisian_call_BS_MC_CV, epsilon_95_CV, 100 * epsilon_95_CV / parisian_call_BS_MC_CV))

<br>

## Step 4: Computation of option prices using importance sampling with exponential tilting

Suppose that a trader wants to compute the prices of European call options $V(0) = E_Q(e^{-rT} (S(T) - K)^+)$ in the exponential NIG model in which $S(t) = S(0) e^{(r - \omega)t + X(t)}$ and $X(t)$ is a so-called NIG (normal-inverse Gaussian) process with parameters $\alpha, \beta, \delta, \mu \in \mathbb{R}$ with $|\beta| < \alpha$. This means that $X(t) \sim \mathrm{NIG}(\alpha, \beta, \delta t, \mu t)$, where $\mathrm{NIG}$ denotes the so-called [NIG distribution](https://en.wikipedia.org/wiki/Normal-inverse_Gaussian_distribution). The constant $\omega$ above is defined by \begin{equation} \omega := \mu + \delta\Big(\sqrt{\alpha^2 - \beta^2} - \sqrt{\alpha^2 - (\beta + 1)^2}\Big). \end{equation}
This exponential NIG model has the advantage that it models logarithmic returns using the NIG distribution, which has much heavier tails than the normal distribution that is used in the Black-Scholes model.

We now want to compute $V(0) = E_Q(e^{-rT} (S(T) - K)^+)$ using Monte Carlo simulation in the NIG model with $\alpha = 1$, $\beta = 0$, $\delta = 0.3$ and $\mu = 0$. The code snippet below draws $N$ simulations of $X \sim \mathrm{NIG}(\alpha \delta T, \beta \delta T, 1, 0)$ with $\alpha = 1$, $\beta = 0$ and $\delta = 0.3$ using the scipy.stats class `norminvgauss`. It uses the numpy random seed 20 for reproducibility. We can then use these simulations to obtain samples from the NIG process $X(T)$ by relying on the fact that $\mathrm{NIG}(\alpha, \beta, \delta T, 0) = \delta T \cdot\mathrm{NIG}(\alpha \delta T, \beta \delta T, 1, 0)$. 

*Useful Python commands*: `N` simulations of the $\mathrm{NIG}(\alpha, \beta, 1, 0)$ distribution can be obtained by the scipy.stats command `norminvgauss(a=alpha, b=beta).rvs(N)`

In [ ]:
N = 10000
alpha = 1
beta = 0
delta = 0.3

np.random.seed(20)
X = delta * T * norminvgauss(a=alpha * delta * T, b=beta * delta * T).rvs(N)

**Task 4.1**: Define a variable `omega` for the constant $\omega$ as specified above. Then, initialize an array `disc_payoffs` that contains the discounted call option payoffs $e^{-rT} \big( S(0) \exp((r - \omega)T + \delta T X_n) - K\big)^+$ with $X_n$ being the NIG distributed samples from the array `X`.

*Useful Python commands*: `np.sqrt`, `np.maximum`, `np.exp`

In [ ]:
omega = ...
disc_payoffs = ...

**Task 4.2**: Use the array `disc_payoffs` to compute the Monte Carlo estimate $\widehat V_N$ for the European call option price $V(0)$ in the NIG model and store it in a variable `call_NIG_MC`. Then, define a variable `epsilon_95` that contains the width $\widehat \varepsilon_N := \smash{1.96 \sqrt{\frac{\widehat \sigma^2_N(f(X))}{N}}}$ of an approximate 95\% confidence interval for $V(0)$, where $\widehat \sigma^2_N(f(X))$ denotes the sample variance of the discounted payoffs from the array `disc_payoffs`. Then, print the variables `call_NIG_MC` and `epsilon_95` as well as the relative percentage error $100\frac{\widehat \varepsilon_N}{\widehat V_N}$.

*Useful Python commands*: `np.mean`, `np.var` `np.sqrt`

In [ ]:
call_NIG_MC = ...
epsilon_95 = ...

print('MC option price: {:.5f}, 95% error: {:.5f} ({:.5f}%)'.format(call_NIG_MC, epsilon_95, 100 * epsilon_95 / call_NIG_MC))

The above relative percentage error for the NIG model option price is rather high. We now want to use importance sampling to reduce that error.

If one wants to approximate the expected value $E[f(X)]$ using Monte Carlo simulation but the $X$ is supported mostly in regions where $f$ is comparatively small or even zero, it can be of advantage to use the importance sampling method, which relies on the estimator $\smash{V_{IS} = \frac{1}{N}\sum_{n=1}^N f(Y_n) \frac{g(Y_n)}{h(Y_n)}}$. Here, $g$ is the pdf of $X$, $h$ is some auxiliary pdf and $Y_1, \dots, Y_N$ are simulations from the pdf $h$. In practice, it is sometimes not clear how to choose a suitable auxiliary pdf $h$. In this case, the so-called *exponential tilting* method (also known as *Esscher transform*) can be used, which parameterizes $h$ via $\smash{h(x) := g(x) \frac{e^{\theta x}}{E(e^{\theta X})}}$ with some parameter $\theta \in \mathbb{R}$. One can then try different choices of the auxiliary parameter $\theta$ to find the one that provides the best variance reduction. The advantage of this choice of $h$ is that the ratio $\smash{\frac{g(x)}{h(x)}}$ used in the importance sampling estimator takes the very simple form $\smash{\frac{g(x)}{h(x)} = \frac{E(e^{\theta X})}{e^{\theta x}}}$.

Now, using exponential tilting on the NIG distribution is fairly easy: If $g$ denotes the pdf of the $\mathrm{NIG}(\alpha, \beta, \delta, \mu)$ distribution, then one can show that $\smash{h(x) := g(x) \frac{e^{\theta x}}{E(e^{\theta X})}}$ is the pdf of the $\mathrm{NIG}(\alpha, \beta + \theta, \delta, \mu)$ distribution as long as $-\alpha - \beta < \theta < \alpha - \beta$. Moreover, the value $E(e^{\theta X})$ is given by $E(e^{\theta X}) = \exp\big(\mu \theta + \delta\big(\sqrt{\alpha^2 - \beta^2} - \sqrt{\alpha^2 - (\beta + \theta)^2}\big)\big)$ for $X \sim \mathrm{NIG}(\alpha, \beta, \delta, \mu)$. In the code snippet above Task 4.1, we simulated random variables $X_n$ from the $\mathrm{NIG}(\alpha \delta T, \beta \delta T, 1, 0)$ distribution, so $g$ is the pdf of the $\mathrm{NIG}(\alpha \delta T, \beta \delta T, 1, 0)$ distribution in our setting.

**Task 4.3**: Set $\theta$ equal to 0.1 to test the exponential tilting importance sampling. Then draw $N = 10\,000$ samples from the $\mathrm{NIG}(\alpha \delta T, \beta \delta T + \theta, 1, 0)$ distribution with $\mu = d$ and store them in an array `Y`. Use the numpy random seed 20 for reproducibility. Then define a variable `mgf` that contains the value $E(e^{\theta X})$ for $X \sim \mathrm{NIG}(\alpha \delta T, \beta \delta T, 1, 0)$.

*Useful Python commands*: `N` simulations of the $\mathrm{NIG}(\alpha, \beta, 1, 0)$ distribution can be obtained by the scipy.stats command `norminvgauss(a=alpha, b=beta).rvs(N)`

In [ ]:
theta = 0.1
...
Y = ...
mgf = ...

**Task 4.4**: Initialize an array `disc_payoffs_IS` that contains the discounted importance sampling "payoffs" $e^{-rT - \theta Y_n}  E(e^{\theta X}) \big( S(0) \exp((r - \omega)T + \delta T Y_n) - K\big)^+$. Then, calculate the importance sampling estimator $V_{IS}$ and store it in a variable `call_NIG_MC_IS`. Define a variable `epsilon_95_IS` that contains the width of an approximate 95% confidence interval $\widehat\varepsilon_{IS}$ for this importance sampling estimator. Then, print the variables `call_NIG_MC_IS` and `epsilon_95_IS` as well as the relative percentage error $100\frac{\widehat \varepsilon_{IS}}{\widehat V_{IS}}$ and interpret your result in comparison to the result from Task 4.2.

In [ ]:
disc_payoffs_IS = ...
call_NIG_MC_IS = ...
epsilon_95_IS = ...

print('IS option price: {:.5f}, 95% error: {:.5f} ({:.5f}%)'.format(call_NIG_MC_IS, epsilon_95_IS, 100 * epsilon_95_IS / call_NIG_MC_IS))

The choice of the auxiliary parameter $\theta = 0.1$ did not lead to a large reduction of the confidence error. We therefore have to find the value of $\theta$ that leads to the largest reduction in variance of the Monte Carlo estimator. On page 57 of the lecture notes, we can see that $\mathrm{Var}(V_{IS}) = E\big( (f(X))^2 \frac{g(X)}{h(X)} - V^2\big)$. Therefore, one should choose the pdf $h$ in importance sampling so that $ E\big( (f(X))^2 \frac{g(X)}{h(X)}\big)$ becomes small. In the case of exponential tilting, this means that the optimal parameter $\theta$ is the one that minimizes the value $E\big( (f(X))^2 e^{- \theta X}\big) E\big( e^{\theta X})$. 

**Task 4.5**: Fill the blank spaces in the code below to perform the minimization of $E\big( (f(X))^2 e^{- \theta X}\big) E\big( e^{\theta X})$

*Useful Python commands*: `np.sqrt`, `np.exp`, `np.maximum`, `np.mean`

In [ ]:
def min_func(theta):
    mgf = ...
    return np.mean(np.exp(-theta * X) * mgf * np.square(np.exp(-r * T) * np.maximum(S0 * np.exp((r - omega) * T + delta * T * X) - K, 0)))

opt_theta = minimize(..., x0=0.1, bounds=((-(alpha + beta) * delta * T, (alpha - beta) * delta * T),)).x
print('Optimal value of theta: {:.6f}'.format(opt_theta[0]))

**Task 4.6**: Repeat Tasks 4.3 and 4.4 but this time using the optimal value of $\theta$ instead of $\theta = 0.1$. Interpret your result in comparison to the results of Tasks 4.2 and 4.4.

In [ ]:
theta = opt_theta
...
Y = ...
mgf = ...

disc_payoffs_IS = ...
call_NIG_MC_IS = ...
epsilon_95_IS = ...

print('IS option price: {:.5f}, 95% error: {:.5f} ({:.5f}%)'.format(call_NIG_MC_IS, epsilon_95_IS, 100 * epsilon_95_IS / call_NIG_MC_IS))

<br>

## Step 5: Bias-free simulation of the Heston model

Consider the stochastic variance process $\gamma$ in the Heston model, which follows the SDE
\begin{equation}
d\gamma(t) = (\kappa - \lambda \gamma(t)) dt + \widetilde \sigma\sqrt{\gamma(t)}  \; dW(t)
\end{equation}
with some Brownian motion $W$. Methods described in the lecture notes like the Euler scheme or the Milshtein scheme can be used to obtain a simulation of a path $(\gamma_0, \gamma_1, \dots, \gamma_m)$ from the variance of the Heston model on an equidistant grid $0 = t_0 < t_1 < \dots < t_m = T$. The disadvantage of using these methods is that they have a *discretization bias* in the sense that the expected values of these simulations slightly differ from the expectations of a theoretical path of the variance process. Instead of using the Euler or Milshtein scheme, our goal is now to exactly simulate a bias-free path  in the sense that each $\gamma_i$ is an exact draw from the true distribution of $\gamma(t_i)$ for any $i \in \{0, \dots, m\}$. This can be achieved in the following way:

Suppose you have already simulated $\gamma_0, \dots, \gamma_i$. Then you can draw $\gamma_{i+1}$ from the conditional distribution of $\gamma(t_{i+1})$ given $\gamma(t_{i}) = \gamma_i$, which can be shown to be that of $Y / (2c)$, where $c = 2\lambda / [(1- e^{-\lambda\Delta t}) \widetilde \sigma^2]$ and $Y$ has a non-central chi-squared distribution with $4\kappa / \widetilde \sigma^2$ degrees of freedom and non-centrality parameter $2c e^{-\lambda \Delta t} \gamma_i$. Here, $\Delta t := t_{i+1} - t_i$.

**Task 5.1**: Initialize an array `gamma` of shape `(N, m + 1)` with $N = 10$ and $m = 10\,000$ containing only zeros, which are later supposed to be filled with the $\gamma$ paths. Set the elements of the first column of `gamma` equal to `gam0`.

*Useful Python commands*: `np.zeros`

In [ ]:
N = 10
m = 10000
dt = T / m

gamma = ...
gamma[:, 0] = ...

**Task 5.2**: Define the variable $c = 2\lambda / [(1- e^{-\lambda\Delta t}) \widetilde \sigma^2]$. Then, use a for-loop to fill the columns of the array `gamma` with the values $\gamma_i = Y / (2c)$. In the $i$-th iteration of the loop, $Y$ has a non-central chi-squared distribution with $4\kappa / \widetilde \sigma^2$ degrees of freedom and non-centrality parameter $2c e^{-\lambda \Delta t} \gamma_{i-1}$. Use the numpy random seed 20 for reproducibility.

*Useful Python commands*: `np.random.noncentral_chisquare`

In [ ]:
c = ...
...
for i in range(1, m + 1):
    Y = ...
    gamma[:, i] = ...

**Task 5.3**: Plot the $N = 10$ paths stored in the array `gamma`.

In [ ]:
...
plt.show()